# Project 2:  Lucid Titanic Sleuthing - Solution Key 

## Part 1: Data Inspection and Preliminary Commentary

In [ ]:
# to be added by Mike Frantz upon grading

## Part 2:  Data Intake and Package Imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
titanic_path = './assets/titanic.csv'
titanic = pd.read_csv(titanic_path)
print "Data Dimensions by Row and Column:", titanic.shape

Data Dimensions by Row and Column: (1309, 14)


In [3]:
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


#### Check that the age column doesn't have any unreasonable values 

In [4]:
# Values look good with exception of nan

titanic['age'].unique()

array([ 29.  ,   0.92,   2.  ,  30.  ,  25.  ,  48.  ,  63.  ,  39.  ,
        53.  ,  71.  ,  47.  ,  18.  ,  24.  ,  26.  ,  80.  ,    nan,
        50.  ,  32.  ,  36.  ,  37.  ,  42.  ,  19.  ,  35.  ,  28.  ,
        45.  ,  40.  ,  58.  ,  22.  ,  41.  ,  44.  ,  59.  ,  60.  ,
        33.  ,  17.  ,  11.  ,  14.  ,  49.  ,  76.  ,  46.  ,  27.  ,
        64.  ,  55.  ,  70.  ,  38.  ,  51.  ,  31.  ,   4.  ,  54.  ,
        23.  ,  43.  ,  52.  ,  16.  ,  32.5 ,  21.  ,  15.  ,  65.  ,
        28.5 ,  45.5 ,  56.  ,  13.  ,  61.  ,  34.  ,   6.  ,  57.  ,
        62.  ,  67.  ,   1.  ,  12.  ,  20.  ,   0.83,   8.  ,   0.67,
         7.  ,   3.  ,  36.5 ,  18.5 ,   5.  ,  66.  ,   9.  ,   0.75,
        70.5 ,  22.5 ,   0.33,   0.17,  40.5 ,  10.  ,  23.5 ,  34.5 ,
        20.5 ,  30.5 ,  55.5 ,  38.5 ,  14.5 ,  24.5 ,  60.5 ,  74.  ,
         0.42,  11.5 ,  26.5 ])

#### Check for missing values.  How do you know that a value is missing?

In [5]:
print titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB
None


## Part 3: data imputation

#### Well let’s say that it does... You likely noticed that Age has some missing values. How many are missing?

In [6]:
print "Missing age values:", titanic['age'].isnull().sum()

Missing age values: 263


#### For the Age of the passengers ... how would you guess at the missing values using the other data present in the CSV.

In [33]:
print "Mean age:", titanic['age'].mean()
print "Median age:", titanic['age'].median()

titanic['imputed_age'] = titanic['age'].apply(lambda x: titanic['age'].median() if np.isnan(x) else x)

# titanic['Age'].fillna(titanic['Age'].median(), inplace=True)

Mean age: 29.8811376673
Median age: 28.0


In [8]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
pclass         1309 non-null int64
survived       1309 non-null int64
name           1309 non-null object
sex            1309 non-null object
age            1046 non-null float64
sibsp          1309 non-null int64
parch          1309 non-null int64
ticket         1309 non-null object
fare           1308 non-null float64
cabin          295 non-null object
embarked       1307 non-null object
boat           486 non-null object
body           121 non-null float64
home.dest      745 non-null object
imputed_age    1309 non-null float64
dtypes: float64(4), int64(4), object(7)
memory usage: 153.5+ KB


In [9]:
def get_titles():

    global titanic
    
    # we extract the title from each name
    titanic['Title'] = titanic['name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    
    # a map of more aggregated titles
    Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"

                        }
    
    titanic['Title'] = titanic.Title.map(Title_Dictionary)
    
get_titles()

In [10]:
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,imputed_age,Title
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",29.00,Miss
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",0.92,Master
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",2.00,Miss
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",30.00,Mr
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",25.00,Mrs


In [11]:
grouped = titanic.groupby(['sex','pclass','Title'])
grouped.median()

survived   age  sibsp  parch      fare   body  \
sex    pclass Title                                                    
female 1      Miss          1.0  30.0    0.0    0.0   99.9625    NaN   
              Mrs           1.0  45.0    1.0    0.0   78.1125    NaN   
              Officer       1.0  49.0    0.0    0.0   25.9292    NaN   
              Royalty       1.0  39.0    0.0    0.0   86.5000    NaN   
       2      Miss          1.0  20.0    0.0    0.0   20.2500    NaN   
              Mrs           1.0  30.0    1.0    0.0   26.0000   52.0   
       3      Miss          0.0  18.0    0.0    0.0    8.0500  299.0   
              Mrs           1.0  31.0    1.0    1.0   15.5000  129.5   
male   1      Master        1.0   6.0    1.0    2.0  134.5000    NaN   
              Mr            0.0  41.5    0.0    0.0   47.1000  157.0   
              Officer       0.5  52.0    0.0    0.0   37.5500  230.0   
              Royalty       0.0  40.0    0.0    0.0   27.7208    NaN   
       2      Master        1.0   2.0    1.0    1.0   26.0000    NaN   
              Mr            0.0  30.0    0.0    0.0   13.0000  155.0   
              Officer       0.0  41.5    0.0    0.0   13.0000  174.0   
       3      Master        0.0   6.0    3.0    1.0   22.3583    4.0   
              Mr            0.0  26.0    0.0    0.0    7.8958  156.0   

                       imputed_age  
sex    pclass Title                 
female 1      Miss            30.0  
              Mrs             43.0  
              Officer         49.0  
              Royalty         39.0  
       2      Miss            20.5  
              Mrs             30.0  
       3      Miss            23.0  
              Mrs             28.0  
male   1      Master           6.0  
              Mr              36.0  
              Officer         51.0  
              Royalty         40.0  
       2      Master           2.0  
              Mr              29.0  
              Officer         41.5  
       3      Master           7.0  
              Mr              28.0

In [12]:
def process_age():
    
    global titanic
    
    # a function that fills the missing values of the Age variable
    
    def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26
    
    titanic.Age = titanic.apply(lambda row : fillAges(row) if np.isnan(row['Age']) else row['Age'], axis=1)

In [13]:
# run process age and revisit group statistics below...
# 

## Part 4: Group Statistics

#### Are there any groups that were especially adversely affected in the Titanic wreck? Are there any groups that outperformed the survival of the latter group? (justify your response numerically)

###### Survival Rate by Sex
Females had a much higher survival rate than males. The p-value is 4.5899249369529454e-81, indicating a statistically significant difference at any reasonable significance level.

In [14]:
# sex_table = survival_pivot('sex', titanic)
titanic.pivot_table('name', index='sex', columns='survived', aggfunc='count', margins=True)

survived,0,1,All
sex,,,
female,127.0,339.0,466.0
male,682.0,161.0,843.0
All,809.0,500.0,1309.0


In [15]:
table = pd.crosstab( titanic['sex'], titanic['survived'] )
table

survived,0,1
sex,,
female,127,339
male,682,161


In [16]:
chi, p, df, expected = chi2_contingency(table)
print "p-value:", p

p-value: 4.58992493695e-81


#### Survival Rate by Passenger Class
There appears to be a strong relationship between class and survival rate: first-class had the highest survival rate, then second, then third.

In [17]:
table = pd.crosstab( titanic['pclass'], titanic['survived'] )
table

survived,0,1
pclass,,
1,123,200
2,158,119
3,528,181


In [18]:
chi, p, df, expected = chi2_contingency(table)
print "p-value:", p

p-value: 1.72082595883e-28


#### Survival Rate by Sex and Class


In [19]:
# pivot table here

#### Survival Rate by Age

In [20]:
# a cut off for adulthood between 14 and 18 would have been appropriate 
# then run chi-squared test as per above


## Part 5:  Comparative Statistics:  Lusitania

#### Get the data

In [21]:
lusitania_path = './assets/lusitania.csv'
lusitania = pd.read_csv(lusitania_path)
print "Data Dimensions by Row and Column:", lusitania.shape

Data Dimensions by Row and Column: (1961, 23)


In [22]:
lusitania.head()

,Family name,Title,Personal name,Fate,Age,Department/Class,Passenger/Crew,Citizenship,Position,Status,...,Country,Lifeboat,Rescue Vessel,Body No.,Ticket No.,Cabin No.,Traveling Companions and other notes,Value,Adult/Minor,Sex
0,CAMERON,Mr.,Charles W.,Lost,38,Band,Crew,British,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adult,Male
1,CARR-JONES,Mr.,E.,Lost,37,Band,Crew,British,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adult,Male
2,DRAKEFORD,Mr.,Edward,Saved,30,Band,Crew,British,Violin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adult,Male
3,HAWKINS,Mr.,Handel,Saved,25,Band,Crew,British,Cello,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adult,Male
4,HEMINGWAY,Mr.,John William,Saved,27,Band,Crew,British,Double Bass,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adult,Male


#### Clean up the Fate column

In [23]:
lusitania['Fate'].value_counts(dropna=False)

Lost                        1193
Saved                        763
Saved (died from trauma)       4
Not on board                   1
Name: Fate, dtype: int64

#### Clean up the Age column

In [24]:
# The lusitania data has some weird stuff going on in the Age column. We'll have to clean that up. 
lusitania['Age'].unique()

array(['38', '37', '30', '25', '27', '48', nan, '24', '19 ?', '57', '50',
       '56', '41', '19', '33', '29', '18', '20', '21', '26', '17', '58',
       '47', '54', '35', '43', '59', '53', '44', '51', '40', '49', '42',
       '32', '31', '34', '22', '45', '36', '29 ?', '52', '23', '60', '28',
       '16', '46', '15', '39', '63 ?', '55', '64', '53 ?', '09-months',
       '6', '9', '14', '10', '12', '?', '62', '5', '8', '65', '68', '76',
       '61', '63', '03-months', '1.5', '2.5', '18-months', '3', '2',
       '25 ?', '08-months', '4', '15-months', '14-months', 'Infant',
       '3-6-months ?', '51 or 47', '45 or 51', '22 (?)', '17-months',
       '13-months', '02-months', '49 ?', '05-months', '11', '59 (64?)',
       '63 (51?)', '37 (39?)', '32 (22?)', '31 (32?)', '24 (23?)', '27?',
       '7', '42?', '31 ?', '10-months', '38 (?)', '70', '62 (?)',
       '06-months', '13', '2_', '11-months', '30 ?', '34?', '1',
       '39 (47?)', '67', '73', '72', '11-months (3?)', '4.25', '69', '26?'

In [25]:
import re

def clean_age_column(age):
    
    if not isinstance(age, basestring):
        return age
    
    case_ = re.search('(\d+)-months', age)
    if case_:
        return int(case_.groups(1)[0]) / 12.
    
    case_ = re.search('(\d+) or (\d+)', age)
    if case_:
        c1, c2 = case_.groups(1)
        return (int(c1) + int(c2)) / 2.
    
    case_ = re.search('(\d+) \((\d+)\?\)', age)
    if case_:
        c1, c2 = case_.groups(1)
        return (int(c1) + int(c2)) / 2.
    
    case_ = re.search('(\d+)\s?\?', age)
    if case_:
        return int(case_.groups(1)[0])
    
    case_ = re.search('(\d+) \(\?\)', age)
    if case_:
        return int(case_.groups(1)[0])
      
    if age == 'Infant':
        return 1
    
    elif age == '2_':
        return 25
    
    elif age == '?':
        return np.nan
    
    else:
        return age

In [26]:
cleaned_age = lusitania['Age'].apply(clean_age_column)
cleaned_age.unique()

array(['38', '37', '30', '25', '27', '48', nan, '24', 19, '57', '50', '56',
       '41', '19', '33', '29', '18', '20', '21', '26', '17', '58', '47',
       '54', '35', '43', '59', '53', '44', '51', '40', '49', '42', '32',
       '31', '34', '22', '45', '36', 29, '52', '23', '60', '28', '16',
       '46', '15', '39', 63, '55', '64', 53, 0.75, '6', '9', '14', '10',
       '12', '62', '5', '8', '65', '68', '76', '61', '63', 0.25, '1.5',
       '2.5', 1.5, '3', '2', 25, 0.6666666666666666, '4', 1.25,
       1.1666666666666667, 1, 0.5, 49.0, 48.0, 22, 1.4166666666666667,
       1.0833333333333333, 0.16666666666666666, 0.4166666666666667, '11',
       61.5, 57.0, 38.0, 27.0, 31.5, 23.5, '7', 42, 31, 0.8333333333333334,
       '70', 62, '13', 0.9166666666666666, 30, 34, '1', 43.0, '67', '73',
       '72', '4.25', '69', 26, 54, 21, 16], dtype=object)

In [27]:
lusitania['Age'] = cleaned_age.astype(float)

In [28]:
print lusitania['Age'].describe(), "\n"
lusitania.info()

count    1307.000000
mean       32.298202
std        14.365629
min         0.166667
25%              NaN
50%              NaN
75%              NaN
max        76.000000
Name: Age, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1961 entries, 0 to 1960
Data columns (total 23 columns):
Family name                             1961 non-null object
Title                                   1961 non-null object
Personal name                           1958 non-null object
Fate                                    1961 non-null object
Age                                     1307 non-null float64
Department/Class                        1961 non-null object
Passenger/Crew                          1961 non-null object
Citizenship                             1959 non-null object
Position                                691 non-null object
Status                                  795 non-null object
City                                    1279 non-null object
County                     

/home/jam/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [29]:
print "Lusitania missing age values:", lusitania['Age'].isnull().sum()

Lusitania missing age values: 654


In [30]:
# please note: you should collapse the not on board and saved died from trauma into another column
table = pd.crosstab(lusitania['Sex'],lusitania['Fate'])
table

Fate,Lost,Not on board,Saved,Saved (died from trauma)
Sex,,,,
Female,325,1,191,1
Male,868,0,572,3


In [31]:
chi, p, df, expected = chi2_contingency(table)
print "p-value:", p

p-value: 0.265833818484


#### Are there any groups that were especially adversely affected in the Lusitania wreck? Are there any groups that outperformed the survival of the latter group? (justify your response numerically)

#### Survival Rate by Sex
Unlike the Titanic, in the Lusitania we cannot reject the null hypothesis that men and women had similar survival rates given the p-value largely above 0.05.



#### Survival Rate by Age



In [32]:
# Also yields a p-value above 0.05

#### In sum, the "women and children first" hypothesis that prevailed for the Titanic did not persist for the Lusitania.  Class was also not a factor (add justificative pivot tables and statistics as necessary)